In [ ]:
# 필요한 패키지 설치
!pip install -U langchain-community
!pip install openai langchain pypdf tiktoken sentence-transformers chromadb
!pip install langchain
!pip install transformers

In [ ]:
# 필요한 라이브러리 임포트
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
import numpy as np
import csv

In [ ]:
# 코사인 유사도 함수 정의
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# 토크나이저 설정
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

## 한국어 사전 학습 모델임베딩 -ko-sbert-nli

In [ ]:
# 오픈 소스 임베딩 함수 생성
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': 'True'}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

## Chroma

In [ ]:
import csv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document  # Document 클래스 임포트

In [ ]:
# CSV 파일 로드 및 문장 텍스트 추출
csv_path = "/content/drive/MyDrive/filtered_df.csv"  # 여기에 CSV 파일 경로를 넣기
sentence_texts = []
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        sentence_texts.append(row[1])  # CSV 파일의 각 행의 두 번째 열에 있는 문장 내용을 리스트에 추가


In [ ]:
# HuggingFaceEmbeddings 인스턴스 생성
hf = HuggingFaceEmbeddings()

# 문장 텍스트를 이용하여 임베딩 생성
sentence_embeddings = hf.embed_documents(sentence_texts)

# Document 객체 생성
documents = [Document(page_content=text) for text in sentence_texts]

# Chroma 데이터베이스에 문서 저장
db = Chroma.from_documents(documents, hf, persist_directory="./chroma_db")

In [ ]:
# 사용자 입력을 받아서 임베딩 생성
user_input = input("질문을 입력하세요: ")  # 실제로는 동적으로 받아야 함

# 문장 텍스트를 이용하여 임베딩 생성
query_embedding = hf.embed_documents([user_input])[0]

# Chroma 데이터베이스 불러오기
db3 = Chroma(persist_directory="./chroma_db", embedding_function=hf)


In [ ]:
# 유사도 계산 및 유사한 문장 검색
docs_with_scores = db3.similarity_search(user_input, k=1)


In [ ]:
# 결과 반환 함수
def retrieve_most_similar_sentence(docs_with_scores):
    if docs_with_scores:
        most_similar_document = docs_with_scores[0]  # 첫 번째 결과의 문서 객체
        return most_similar_document.page_content  # 문장 내용 반환
    else:
        return None

In [ ]:
# 리트리버에게 반환할 값
most_similar_sentence = retrieve_most_similar_sentence(docs_with_scores)
print("가장 유사한 문장:", most_similar_sentence)